In [2]:
import tensorflow as tf

In [3]:
tf.compat.v1.Session()

2024-04-23 09:50:22.043826: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-23 09:50:22.063399: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
#import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.metrics import mean_squared_error
import random
import pickle

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
from scipy.stats.distributions import uniform, skewnorm
import keras

from sklearn.metrics import accuracy_score
# import packages for hyperparameters tuning
#import hyperopt
#from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import tensorflow as tf
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Input, Dense, LeakyReLU, PReLU,Dropout
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import ParameterSampler
shuffle_on= True
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [12]:
##################### Calling the dataframe and preparing the data ##########################
root_df = './'
root_models = './'

df01=pd.read_pickle(root_df+"dataframe_surrogateopt_randscan.pickle")

# Deleting no injection configurations
df01['dE_mad'] = df01['dE_mad'].replace([0], np.nan)
df01 = df01[df01['dE_mad'].notna()]

# Choose needed columns (if take all columns scaler will have an error as some columns not numeric)
df01 = df01[["x_of", "a_0", "c_N2","p_1","E_med","dE_mad","q_end","n_emit_y"]]
df = df01.copy()

print(len(df))

3756


In [13]:
# Data resclaing
scaler = MinMaxScaler() #MinMax works very good if borders of inputs in different data sets at least close to each other
df_transformed = scaler.fit_transform(df)
df = pd.DataFrame(df_transformed,
                       columns=df.columns.values.tolist())

#### Saving rescale in pickle ####
#pickle.dump(scaler, open(root_df +"scaler.pkl","wb"))

#### To open it use ####
#scaler_reload = pickle.load(open("scaler.pkl",'rb'))


In [14]:
keras.__version__

'3.2.0'

In [15]:
################### splitting the data between train aand test #######################
x_train, x_test, y_train, y_test = train_test_split(df[["x_of", "a_0", "c_N2","p_1"]],
                                                    df[["E_med","dE_mad","q_end","n_emit_y"]],
                                                    test_size=0.15,shuffle= False,random_state=777)


# Define the number of folds for cross-validation
k_folds = 10

#Putting the train data to the proper format
X_train=np.asarray(x_train).astype('float32')

y_train=np.asarray(y_train).astype('float32')
print(len(X_train))

3192


In [9]:
############ Training the model##################

act1 = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None,
    shared_axes=None)
act2 = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None,
    shared_axes=None)
act3 = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None,
    shared_axes=None)
# Create the model
model = Sequential()
# Add the layers
model.add(Input(shape=(4,)))
model.add(Dense(64,  activation=None))
model.add(act1)
model.add(Dropout(0.2))

model.add(Dense(64,  activation=None))
model.add(act2)
model.add(Dropout(0.2))

model.add(Dense(64,  activation=None))
model.add(act3)
model.add(Dropout(0.2))
#### Output layer for not scaled data
#model.add(Dense(4, activation='linear',kernel_constraint=keras.constraints.NonNeg()))
#### Output layer for scaled data
model.add(Dense(4, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu (PReLU)                 │ (None, 64)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_1 (PReLU)               │ (None, 64)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_2 (PReLU)               │ (None, 64)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,092 (35.52 KB)

 Trainable params: 9,092 (35.52 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Define the K-fold cross-validator
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
# Initialize the list to store the evaluation results for each fold
scores = []
# Define the callback to save the best model
checkpoint = ModelCheckpoint(root_models+'nn/'+'test_rescaled.keras', monitor='val_loss', save_best_only=True)

# Perform K-fold cross-validation
for train_index, val_index in kfold.split(X_train):

  # Get the training and validation data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Fit the model to the training data for this fold
    model.fit(X_train_fold, y_train_fold, epochs=200, batch_size=50,verbose=0, callbacks=[checkpoint], validation_data=(X_val_fold, y_val_fold))

    # Evaluate the model on the validation data for this fold
    score = model.evaluate(X_val_fold, y_val_fold, verbose=1)

    # Add the score for this fold to the list of scores
    scores.append(score)

# Print the mean and standard deviation of the scores across all folds
print("Mean score: %.7f (%.7f)" % (np.mean(scores), np.std(scores)))

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - loss: 0.0011
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 496us/step - loss: 0.0011
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - loss: 0.0012
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step - loss: 0.0011   
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - loss: 8.8041e-04
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 529us/step - loss: 0.0010
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step - loss: 7.6013e-04
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 577us/step - loss: 0.0010   
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - loss: 7.7618e-04
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 535us/step - loss: 8.1894e-04
Mean score: 0.0010079 (0.0001453)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu (PReLU)                 │ (None, 64)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_1 (PReLU)               │ (None, 64)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_2 (PReLU)               │ (None, 64)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,278 (106.56 KB)

 Trainable params: 9,092 (35.52 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 18,186 (71.04 KB)

In [13]:
model.predict(x_test.to_numpy())

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


array([[1.32396713e-01, 5.99293530e-01, 2.31300250e-01, 2.56618351e-01],
       [4.23482597e-01, 1.65872350e-02, 1.13714393e-02, 2.96142563e-04],
       [1.68980718e-01, 4.60453779e-01, 2.98764855e-01, 2.50851601e-01],
       ...,
       [3.35579664e-01, 1.69979986e-02, 1.06526064e-02, 7.94807929e-05],
       [9.48645249e-02, 6.59779787e-01, 2.76358634e-01, 5.47771692e-01],
       [6.90223992e-01, 5.68523183e-02, 1.04373194e-01, 8.28522220e-02]],
      dtype=float32)

In [14]:
#### Model prediction 

nn_model = load_model(root_models+'nn/'+'test_rescaled.keras',compile=False)
y_pred1_test = nn_model.predict(x_test.to_numpy())
y_pred_train = nn_model.predict(x_train.to_numpy())

r2_NN = r2_score(y_test.to_numpy(), y_pred1_test)
print('Test NN :',r2_NN)

r2_NN = r2_score(y_train, y_pred_train)
print('Train NN :',r2_NN)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step
Test NN : 0.9804225100971873
Train NN : 0.9833921742327566


In [15]:
nn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu (PReLU)                 │ (None, 64)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_1 (PReLU)               │ (None, 64)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_2 (PReLU)               │ (None, 64)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,092 (35.52 KB)

 Trainable params: 9,092 (35.52 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
### Checking on other dataframe. Can be deleted

df02=pd.read_pickle(root_df+"dataframe_fulldf2")
df02.rename(columns = {'E_mad':'dE_mad', 'emit_y':'n_emit_y'}, inplace = True) 
df02['dE_mad'] = df02['dE_mad'].replace([0], np.nan)
df02 = df02[df02['dE_mad'].notna()]
df02 = df02[["x_of", "a_0", "c_N2","p_1","E_med","dE_mad","q_end","n_emit_y"]]

scaler1 = MinMaxScaler() 
df_transformed2 = scaler1.fit_transform(df02)
df02 = pd.DataFrame(df_transformed2,
                       columns=df.columns.values.tolist())

x=df02[["x_of", "a_0", "c_N2","p_1"]].to_numpy()
y=df02[["E_med","dE_mad","q_end","n_emit_y"]].to_numpy()
df02_pred = nn_model.predict(x)
r2_NN = r2_score(y, df02_pred)
print('R2 NN :',r2_NN)

344/344 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step
R2 NN : 0.9659718458204444


In [1]:
import tf2onnx
import onnx
import onnxruntime as ort

2024-04-23 09:50:06.900966: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
#prepare the input format
input_signature=[tf.TensorSpec([None, 4], tf.float64, name='input')]
#output must have a name, otherwise the convertion will not work
model.output_names=['output']

In [ ]:
#convert from keras model (model) to onnx, save the onnx model into a file
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13, output_path='model2.onnx')

In [6]:
# Load the ONNX model from the file
session = ort.InferenceSession('model2.onnx')

In [7]:
# to get the name of input (optional)
input_name= session.get_inputs()[0].name
input_name

'input'

In [8]:
# to get the shape of inputs (optional)
input_shape = session.get_inputs()[0].shape
input_shape

['unk__19', 4]

In [16]:
# our test set
x_test.to_numpy()

array([[0.04414279, 0.03696395, 0.14718525, 0.92301997],
       [0.98181373, 0.37218034, 0.6189666 , 0.19693942],
       [0.2311267 , 0.1554619 , 0.53956506, 0.61822893],
       ...,
       [0.90003744, 0.62109482, 0.49487012, 0.0943088 ],
       [0.70989143, 0.06717811, 0.68066123, 0.97240594],
       [0.73405388, 0.32554836, 0.07308455, 0.57993409]])

In [23]:
#predict with the keras model
model.predict(x_test.to_numpy())

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step


array([[1.32396713e-01, 5.99293530e-01, 2.31300250e-01, 2.56618351e-01],
       [4.23482597e-01, 1.65872350e-02, 1.13714393e-02, 2.96142563e-04],
       [1.68980718e-01, 4.60453779e-01, 2.98764855e-01, 2.50851601e-01],
       ...,
       [3.35579664e-01, 1.69979986e-02, 1.06526064e-02, 7.94807929e-05],
       [9.48645249e-02, 6.59779787e-01, 2.76358634e-01, 5.47771692e-01],
       [6.90223992e-01, 5.68523183e-02, 1.04373194e-01, 8.28522220e-02]],
      dtype=float32)

In [17]:
#predict the same with onnx model
#you need to rerun entire notebook to get the same results as for keras
input_data=x_test.to_numpy()
session.run(None, {input_name: input_data})

[array([[1.3161433e-01, 6.1860895e-01, 2.3654518e-01, 2.4215820e-01],
        [3.9357281e-01, 1.5020788e-02, 7.8806579e-03, 1.1456013e-04],
        [1.7180309e-01, 4.5905560e-01, 3.1645855e-01, 2.4086457e-01],
        ...,
        [3.0946493e-01, 1.7964244e-02, 6.3628554e-03, 3.3259392e-05],
        [1.0175699e-01, 7.0444536e-01, 3.0819720e-01, 5.7585794e-01],
        [6.7588967e-01, 6.7409754e-02, 9.3403339e-02, 9.3432277e-02]],
       dtype=float32)]